# Import pour le code en général

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import timedelta
import movingpandas as mpd
import hvplot.pandas
import json
import contextily as ctx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pykalman
from sqlalchemy import text
from sqlalchemy import create_engine

# Chargement des fichiers liés aux données gps du participants

In [2]:
# Charger les fichiers CSV
activity_df = pd.read_csv("../participant-data-semain43/Activities/Participant9999915-activities.csv", encoding='latin1', on_bad_lines='skip')
print("1. Données des activités chargées :\n", activity_df.head())

gps_log_df = pd.read_csv(
    "../participant-data-semain43/GPS/Participant9999915-gps.csv", encoding='latin1', on_bad_lines='skip')
print("2. Données GPS chargées :\n", gps_log_df.head())

sensor_measures_df = pd.read_csv("../participant-data-semain43/Measures/Participant9999915-measures.csv", encoding='latin1', on_bad_lines='skip')
print("3. Données des capteurs chargées :\n", sensor_measures_df.head())

1. Données des activités chargées :
    participant_virtual_id                    time  activity
0                 9999915  2019-10-19 09:53:41+00  Domicile
1                 9999915  2019-10-19 14:27:32+00       Rue
2                 9999915  2019-10-19 14:38:45+00   Magasin
3                 9999915  2019-10-19 14:58:32+00       Rue
4                 9999915  2019-10-19 15:06:57+00  Domicile
2. Données GPS chargées :
    participant_virtual_id               timestamp        lat       lon
0                 9999915  2019-10-19 09:23:32+00  48.801358  2.130643
1                 9999915  2019-10-19 09:24:03+00  48.801383  2.130557
2                 9999915  2019-10-19 09:24:34+00  48.801385  2.130562
3                 9999915  2019-10-19 09:25:05+00  48.801510  2.130485
4                 9999915  2019-10-19 09:25:36+00  48.801810  2.129938
3. Données des capteurs chargées :
    participant_virtual_id                    time  PM2.5  PM10  PM1.0  \
0                 9999915  2019-10-19 10:

# Modification des données par jour 

In [11]:
gps_log_df['timestamp'] = pd.to_datetime(gps_log_df['timestamp'], errors='coerce')
activity_df['time'] = pd.to_datetime(activity_df['time'], errors='coerce')
sensor_measures_df['time'] = pd.to_datetime(sensor_measures_df['time'], errors='coerce')
# Supprimer les fuseaux horaires pour éviter les erreurs
gps_log_df['timestamp'] = gps_log_df['timestamp'].dt.tz_localize(None)
activity_df['time'] = activity_df['time'].dt.tz_localize(None)
sensor_measures_df['time'] = sensor_measures_df['time'].dt.tz_localize(None)
print("4. Fuseaux horaires supprimés des données")
print("2. Données GPS chargées :\n", gps_log_df.head())
print("2. Données activite chargées :\n", activity_df.head())
print("2. Données sensor chargées :\n", sensor_measures_df.head())

4. Fuseaux horaires supprimés des données
2. Données GPS chargées :
    participant_virtual_id           timestamp        lat       lon        date
0                 9999915 2019-10-19 09:23:32  48.801358  2.130643  2019-10-19
1                 9999915 2019-10-19 09:24:03  48.801383  2.130557  2019-10-19
2                 9999915 2019-10-19 09:24:34  48.801385  2.130562  2019-10-19
3                 9999915 2019-10-19 09:25:05  48.801510  2.130485  2019-10-19
4                 9999915 2019-10-19 09:25:36  48.801810  2.129938  2019-10-19
2. Données activite chargées :
    participant_virtual_id                time  activity        date
0                 9999915 2019-10-19 09:53:41  Domicile  2019-10-19
1                 9999915 2019-10-19 14:27:32       Rue  2019-10-19
2                 9999915 2019-10-19 14:38:45   Magasin  2019-10-19
3                 9999915 2019-10-19 14:58:32       Rue  2019-10-19
4                 9999915 2019-10-19 15:06:57  Domicile  2019-10-19
2. Données sensor

In [12]:
# 🔹 Ajouter une colonne 'date' en extrayant uniquement YYYY-MM-DD
gps_log_df["date"] = gps_log_df["timestamp"].dt.date
activity_df["date"] = activity_df["time"].dt.date
sensor_measures_df["date"] = sensor_measures_df["time"].dt.date

# 🔹 Vérifier les premières valeurs
print("Données GPS segmentées par jour :\n", gps_log_df.head())
print("Données d'activités segmentées par jour :\n", activity_df.head())
print("Données capteurs segmentées par jour :\n", sensor_measures_df.head())

# 🔹 Grouper les données par jour (optionnel)
gps_grouped = gps_log_df.groupby("date")
activity_grouped = activity_df.groupby("date")
sensor_grouped = sensor_measures_df.groupby("date")

# 🔹 Afficher un aperçu des groupes
print(f"Nombre de jours distincts dans les données GPS : {len(gps_grouped)}")
print(f"Nombre de jours distincts dans les activités : {len(activity_grouped)}")
print(f"Nombre de jours distincts dans les capteurs : {len(sensor_grouped)}")

Données GPS segmentées par jour :
    participant_virtual_id           timestamp        lat       lon        date
0                 9999915 2019-10-19 09:23:32  48.801358  2.130643  2019-10-19
1                 9999915 2019-10-19 09:24:03  48.801383  2.130557  2019-10-19
2                 9999915 2019-10-19 09:24:34  48.801385  2.130562  2019-10-19
3                 9999915 2019-10-19 09:25:05  48.801510  2.130485  2019-10-19
4                 9999915 2019-10-19 09:25:36  48.801810  2.129938  2019-10-19
Données d'activités segmentées par jour :
    participant_virtual_id                time  activity        date
0                 9999915 2019-10-19 09:53:41  Domicile  2019-10-19
1                 9999915 2019-10-19 14:27:32       Rue  2019-10-19
2                 9999915 2019-10-19 14:38:45   Magasin  2019-10-19
3                 9999915 2019-10-19 14:58:32       Rue  2019-10-19
4                 9999915 2019-10-19 15:06:57  Domicile  2019-10-19
Données capteurs segmentées par jour :
 

# Transformation des données en dataframe Geopanda

In [13]:
# Create a GeoDataFrame 
gps_log_df['geometry'] = [Point(xy) for xy in zip(gps_log_df['lon'], gps_log_df['lat'])]
gdf = gpd.GeoDataFrame(gps_log_df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)  # Assuming the coordinates are in WGS84


,participant_virtual_id,timestamp,lat,lon,date,geometry
0,9999915,2019-10-19 09:23:32,48.801358,2.130643,2019-10-19,POINT (2.13064 48.80136)
1,9999915,2019-10-19 09:24:03,48.801383,2.130557,2019-10-19,POINT (2.13056 48.80138)
2,9999915,2019-10-19 09:24:34,48.801385,2.130562,2019-10-19,POINT (2.13056 48.80138)
3,9999915,2019-10-19 09:25:05,48.801510,2.130485,2019-10-19,POINT (2.13048 48.80151)
4,9999915,2019-10-19 09:25:36,48.801810,2.129938,2019-10-19,POINT (2.12994 48.80181)
...,...,...,...,...,...,...
13626,9999915,2019-10-26 09:13:14,48.799100,2.127310,2019-10-26,POINT (2.12731 48.7991)
13627,9999915,2019-10-26 09:13:45,48.799953,2.128147,2019-10-26,POINT (2.12815 48.79995)
13628,9999915,2019-10-26 09:14:16,48.801183,2.129197,2019-10-26,POINT (2.1292 48.80118)
13629,9999915,2019-10-26 09:14:47,48.801730,2.130540,2019-10-26,POINT (2.13054 48.80173)


In [14]:
#Créer des points géométriques pour les données GPS
geometry = [Point(xy) for xy in zip(gps_log_df['lon'], gps_log_df['lat'])]
gdf = gpd.GeoDataFrame(gps_log_df, geometry=geometry, crs="EPSG:4326")

#Ajouter un identifiant de trajectoire basé sur l'utilisateur ou une logique arbitraire
if 'user_id' in gdf.columns:
    gdf['trajectory_id'] = gdf['user_id']
else:
    gdf['trajectory_id'] = gdf.index // 100  # Exemple : diviser en blocs arbitraires

#Transformation en TrajectoryCollection avec MovingPandas
trajectory_collection = mpd.TrajectoryCollection(gdf, traj_id_col='trajectory_id', t='timestamp')

#Validation des trajectoires créées
print(f"Nombre de trajectoires détectées : {len(trajectory_collection)}")
for traj in trajectory_collection:
    print(traj)

#Visualisation des trajectoires pour validation
map_visualization = gdf.hvplot(geo=True, tiles='OSM', c='trajectory_id', line_width=2, width=800, height=600)
map_visualization

Nombre de trajectoires détectées : 137
Trajectory 0 (2019-10-19 09:23:32 to 2019-10-19 10:14:32) | Size: 100 | Length: 7989.8m
Bounds: (2.04186166666667, 48.7989033333333, 2.13064333333333, 48.815445)
LINESTRING (2.13064333333333 48.8013583333333, 2.13055666666667 48.8013833333333, 2.13056166666667 4
Trajectory 1 (2019-10-19 10:15:03 to 2019-10-19 11:06:10) | Size: 100 | Length: 70.9m
Bounds: (2.04560166666667, 48.8152633333333, 2.04578166666667, 48.8154133333333)
LINESTRING (2.045735 48.815395, 2.045735 48.815395, 2.045735 48.815395, 2.045735 48.815395, 2.045735
Trajectory 2 (2019-10-19 11:06:41 to 2019-10-19 11:57:40) | Size: 100 | Length: 64.3m
Bounds: (2.04561333333333, 48.8153, 2.04576666666667, 48.8154166666667)
LINESTRING (2.045725 48.8154, 2.04561333333333 48.815345, 2.04561333333333 48.815345, 2.045613333333
Trajectory 3 (2019-10-19 11:58:11 to 2019-10-19 12:49:18) | Size: 100 | Length: 67.1m
Bounds: (2.04566166666667, 48.81533, 2.04579333333333, 48.81546)
LINESTRING (2.045693

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (trajectory_id)

In [15]:
# Assume 'trajectory_collection' is already created and contains trajectories
for trajectory in trajectory_collection:
    print(trajectory)
    # Adding distance in default units (meters) and custom units (kilometers and yards)
    trajectory.add_distance(overwrite=True, name="distance_m", units="m")
    trajectory.add_distance(overwrite=True, name="distance_km", units="km")
    trajectory.add_distance(overwrite=True, name="distance_yards", units="yd")

    # Adding speed in default units (m/s) and custom units (feet/minute and knots)
    trajectory.add_speed(overwrite=True, name="speed_kmph", units=("km", "h"))
    trajectory.add_speed(overwrite=True, name="speed_ft_min", units=("ft", "min"))
    trajectory.add_speed(overwrite=True, name="speed_knots", units=("nm", "h"))

    # Correctly adding acceleration with appropriate time units
    trajectory.add_acceleration(overwrite=True, name="acceleration_mps2", units=("m", "s", "s"))
    trajectory.add_acceleration(overwrite=True, name="acceleration_mph_s", units=("mi", "h", "s"))


    # Print the updated dataframe to see the added columns
    print(trajectory.df.head())

#Visualisation des trajectoires pour validation
map_visualization = gdf.hvplot(geo=True, tiles='OSM', c='trajectory_id', line_width=2, width=800, height=600)
map_visualization


Trajectory 0 (2019-10-19 09:23:32 to 2019-10-19 10:14:32) | Size: 100 | Length: 7989.8m
Bounds: (2.04186166666667, 48.7989033333333, 2.13064333333333, 48.815445)
LINESTRING (2.13064333333333 48.8013583333333, 2.13055666666667 48.8013833333333, 2.13056166666667 4
                     participant_virtual_id        lat       lon        date  \
timestamp                                                                      
2019-10-19 09:23:32                 9999915  48.801358  2.130643  2019-10-19   
2019-10-19 09:24:03                 9999915  48.801383  2.130557  2019-10-19   
2019-10-19 09:24:34                 9999915  48.801385  2.130562  2019-10-19   
2019-10-19 09:25:05                 9999915  48.801510  2.130485  2019-10-19   
2019-10-19 09:25:36                 9999915  48.801810  2.129938  2019-10-19   

                                     geometry  trajectory_id  distance_m  \
timestamp                                                                  
2019-10-19 09:23:32  POI

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (trajectory_id)

# Nettoyage des données

In [18]:
# Nettoyage des trajectoires avec suppression des outliers
cleaned_trajectories = []
nb_trajectoires_initiales = len(trajectory_collection.trajectories)

for trajectory in trajectory_collection.trajectories:  
    cleaner = mpd.OutlierCleaner(trajectory)  
    cleaned_traj = cleaner.clean(alpha=2)  # Alpha définit la tolérance aux outliers
    
    # Vérifier si la trajectoire nettoyée contient encore des données
    if cleaned_traj is not None and not cleaned_traj.df.empty:
        cleaned_traj.add_speed(overwrite=True, units=("km", "h"))  # Recalculer la vitesse
        cleaned_trajectories.append(cleaned_traj)
        print(f"✅ Trajectoire {trajectory.id} nettoyée : {len(trajectory.df)} → {len(cleaned_traj.df)} points restants")
    else:
        print(f"❌ Trajectoire {trajectory.id} supprimée après nettoyage (trop d'outliers)")

# Vérification du nombre de trajectoires conservées
nb_trajectoires_nettoyees = len(cleaned_trajectories)
print(f"\n📌 Nombre de trajectoires avant nettoyage : {nb_trajectoires_initiales}")
print(f"📌 Nombre de trajectoires après nettoyage : {nb_trajectoires_nettoyees}")

# Création d'une nouvelle collection avec les trajectoires nettoyées
cleaned_trajectory_collection = mpd.TrajectoryCollection(cleaned_trajectories)

# Visualisation interactive des trajectoires nettoyées
gdf_cleaned = gpd.GeoDataFrame(pd.concat([t.df for t in cleaned_trajectories]), crs="EPSG:4326")
map_cleaned = gdf_cleaned.hvplot(
    geo=True, tiles='OSM', c='trajectory_id', line_width=2, width=800, height=600
)
map_cleaned

✅ Trajectoire 0 nettoyée : 100 → 100 points restants
✅ Trajectoire 1 nettoyée : 100 → 100 points restants
✅ Trajectoire 2 nettoyée : 100 → 100 points restants
✅ Trajectoire 3 nettoyée : 100 → 100 points restants
✅ Trajectoire 4 nettoyée : 100 → 100 points restants
✅ Trajectoire 5 nettoyée : 100 → 100 points restants
✅ Trajectoire 6 nettoyée : 100 → 100 points restants
✅ Trajectoire 7 nettoyée : 100 → 100 points restants
✅ Trajectoire 8 nettoyée : 100 → 100 points restants
✅ Trajectoire 9 nettoyée : 100 → 100 points restants
✅ Trajectoire 10 nettoyée : 100 → 100 points restants
✅ Trajectoire 11 nettoyée : 100 → 100 points restants
✅ Trajectoire 12 nettoyée : 100 → 99 points restants
✅ Trajectoire 13 nettoyée : 100 → 100 points restants
✅ Trajectoire 14 nettoyée : 100 → 99 points restants
✅ Trajectoire 15 nettoyée : 100 → 99 points restants
✅ Trajectoire 16 nettoyée : 100 → 100 points restants
✅ Trajectoire 17 nettoyée : 100 → 100 points restants
✅ Trajectoire 18 nettoyée : 100 → 100 poi

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (trajectory_id)

# Créer un dataframe avec les trajectoires sous forme de données géospatiales

In [20]:
import geopandas as gpd

# Créer une liste pour stocker les informations des trajectoires
trajectories_data = []

# Parcourir chaque trajectoire dans la collection
for idx, traj in enumerate(trajectory_collection):
    # Extraire l'identifiant et la géométrie
    trajectory_id = idx  # Identifiant unique basé sur l'index
    geometry = traj.to_linestring()  # Convertir la trajectoire en LINESTRING
    
    # Ajouter les informations à la liste
    trajectories_data.append({
        'id': trajectory_id,
        'geometry': geometry
    })

# Créer un GeoDataFrame à partir de la liste
trajectories_gdf = gpd.GeoDataFrame(trajectories_data, geometry='geometry', crs="EPSG:4326")

# Afficher le résultat
print(trajectories_gdf.head())

   id                                           geometry
0   0  LINESTRING (2.13064 48.80136, 2.13056 48.80138...
1   1  LINESTRING (2.04574 48.8154, 2.04574 48.8154, ...
2   2  LINESTRING (2.04572 48.8154, 2.04561 48.81534,...
3   3  LINESTRING (2.04569 48.81542, 2.04569 48.81542...
4   4  LINESTRING (2.04575 48.81546, 2.04575 48.81546...


# Enregistrer les données pour être utilisable par le code my-home-is-my-secret

In [21]:
trajectories_gdf.to_file("../my-home-is-my-secret/my-home-is-my-secret-master/input/trajectory_versailles.shp")